In [168]:
import pandas as pd
import xml.etree.ElementTree as ET
from sqlalchemy import create_engine

trees = []
roots = []

trees.append(ET.parse('matchstats/Burillo-Podoroska.xml'))
trees.append(ET.parse('matchstats/Danilia-Lodikova.xml'))
trees.append(ET.parse('matchstats/Seidel-Friedsam.xml'))
trees.append(ET.parse('matchstats/Tiomfeeva-Eikeri.xml'))
trees.append(ET.parse('matchstats/Vickery-Collins.xml'))

for tree in trees:
    roots.append(tree.getroot())

In [113]:
def getMatchInfo(root: ET.Element, match_df: pd.DataFrame):
    """
    Parses the matchInfo and round attributes from the XML file and appends them to the match_df
    Args:
        root (ET.Element): root of the XML file for the specific match to parse
        match_df (pd.DataFrame): df to append the matchInfo and round attributes to
    Returns:
        match_df (pd.DataFrame): df with the matchInfo and round attributes appended
    """
    matchInfo = root.find('matchInfo')
    roundInfo = matchInfo.find('round')

    # MatchInfo attribute
    match_df['match_id'] = matchInfo.get('id')
    match_df['match_date'] = matchInfo.get('officialStartDate')
    match_df['match_draw_position'] = matchInfo.get('drawPosition')

    # Round attribute
    match_df['round_id'] = roundInfo.get('id')
    match_df['round_title'] = roundInfo.get('title')
    match_df['round_start_date'] = roundInfo.get('startDate')
    match_df['round_end_date'] = roundInfo.get('endDate')
    match_df['round_scoring_system'] = roundInfo.get('scoringSystem')
    match_df['round_group'] = int(roundInfo.get('group'))
    match_df['round_ties'] = int(roundInfo.get('ties'))
    match_df['round_order_method'] = roundInfo.get('orderMethod')
    match_df['round_prize_money'] = int(roundInfo.get('prizeMoney'))

    return match_df

In [94]:
def getTournamentInfo(root: ET.Element, tournament_df: pd.DataFrame, match_df: pd.DataFrame):
    """
    Parses the tournamentInfo, competitionInfo and tourCalendarInfo attributes from the XML file
    Args:
        root (ET.Element): root of the XML file for the specific match to parse
        tournament_df (pd.DataFrame): df to append tournament attributes to
    Returns:
        tournament_df (pd.DataFrame): df with the tournament attributes appended
        match_df (pd.DataFrame): df with the tournament attributes appended
    """
    matchInfo = root.find('matchInfo')
    tournamentInfo = matchInfo.find('tournament')
    competitionInfo = matchInfo.find('competition')
    tourCalendarInfo = matchInfo.find('tourCalendar')

    # Tournament attribute
    tournament_df['tournament_id'] = tournamentInfo.get('id')
    tournament_df['tournament_name'] = tournamentInfo.get('name')
    tournament_df['tournament_startDate'] = tournamentInfo.get('startDate')
    tournament_df['tournament_endDate'] = tournamentInfo.get('endDate')
    tournament_df['tournament_type'] = tournamentInfo.get('type')
    tournament_df['tournament_drawSize'] = tournamentInfo.get('drawSize')
    tournament_df['tournament_matchType'] = tournamentInfo.get('matchType')
    tournament_df['tournament_gender'] = tournamentInfo.get('gender')
    tournament_df['tournament_status'] = tournamentInfo.get('status')
    tournament_df['tournament_cityName'] = tournamentInfo.get('cityName')
    tournament_df['tournament_winningPeopleId'] = tournamentInfo.get('winningPeopleId')
    tournament_df['tournament_winningPeople'] = tournamentInfo.get('winningPeople')
    tournament_df['tournament_totalPrizeMoney'] = int(tournamentInfo.get('totalPrizeMoney'))
    tournament_df['tournament_totalPrizeCurrency'] = tournamentInfo.get('totalPrizeCurrency')
    tournament_df['tournament_winnerPrizeMoney'] = int(tournamentInfo.get('winnerPrizeMoney'))

    # Competition attribute
    tournament_df['competition_id'] = competitionInfo.get('id')
    tournament_df['competition_name'] = competitionInfo.get('name')
    tournament_df['competition_areaId'] = competitionInfo.get('areaId')
    tournament_df['competition_areaName'] = competitionInfo.get('areaName')
    tournament_df['competition_court'] = competitionInfo.get('court')

    # TourCalendar attribute
    tournament_df['tourCalendar_id'] = tourCalendarInfo.get('id')
    tournament_df['tourCalendar_startDate'] = tourCalendarInfo.get('startDate')
    tournament_df['tourCalendar_endDate'] = tourCalendarInfo.get('endDate')
    tournament_df['tourCalendar_name'] = tourCalendarInfo.text

    # Store selected atrributes from above so that they can be seen without table joins
    match_df['tournament_id'] = tournamentInfo.get('id')
    match_df['competition_id'] = competitionInfo.get('id')
    match_df['tourCalendar_id'] = tourCalendarInfo.get('id')
    match_df['tournament_name'] = tournamentInfo.get('name')
    match_df['court'] = competitionInfo.get('court')
    match_df['matchType'] = tournamentInfo.get('matchType')
    match_df['gender'] = tournamentInfo.get('gender')
    
    return tournament_df, match_df

In [118]:
def getMatchStats(root: ET.Element, match_df: pd.DataFrame):
    """
    Collect matchs stats for each player and store in dataframe
    Args:
        root (ET.Element): root of the XML file for the specific match to parse
        match_df (pd.DataFrame): df that represents a new row
    Returns:
        match_df (pd.DataFrame): new single row df with match stats
    """

    for stats in root.iter('stats'):
        player = stats.get('position').upper()
        
        match_df[f"player{player}_id"] = stats.get('contestantId')
        match_df[f"player{player}_firstName"] = stats.get('firstName')
        match_df[f"player{player}_lastName"] = stats.get('lastName')
        
        for child in stats.iter('stat'):
            # playerA_aces: 5
            match_df[f"player{player}_{child.attrib.get('type')}"] = float(child.text)

    return match_df

In [139]:
def getScore(root: ET.Element, match_df: pd.DataFrame):
    liveData = root.find('liveData')
    matchDetails = liveData.find('matchDetails')
    scoreInfo = matchDetails.find('scores')

    match_df['winner'] = matchDetails.get('winner')
    match_df['matchSet'] = matchDetails.get('matchSet')
    match_df['firstServerId'] = matchDetails.get('firstServerId')

    # If there were no tie breaks or match only went to 2/3/4 or match was forfeited, then the corresponding values will be None
    # The fields are needed for completeness
    match_df['playerA_set1'] = scoreInfo.find('set1').get('a') if scoreInfo.find('set1') is not None else None
    match_df['playerB_set1'] = scoreInfo.find('set1').get('b') if scoreInfo.find('set1') is not None else None
    match_df['playerA_tiebreak1'] = scoreInfo.find('set1').get('tiebreakA') if scoreInfo.find('set1') is not None else None
    match_df['playerB_tiebreak1'] = scoreInfo.find('set1').get('tiebreakB') if scoreInfo.find('set1') is not None else None

    match_df['playerA_set2'] = scoreInfo.find('set2').get('a') if scoreInfo.find('set2') is not None else None
    match_df['playerB_set2'] = scoreInfo.find('set2').get('b') if scoreInfo.find('set2') is not None else None
    match_df['playerA_tiebreak2'] = scoreInfo.find('set2').get('tiebreakA') if scoreInfo.find('set2') is not None else None
    match_df['playerB_tiebreak2'] = scoreInfo.find('set2').get('tiebreakB') if scoreInfo.find('set2') is not None else None

    match_df['playerA_set3'] = scoreInfo.find('set3').get('a') if scoreInfo.find('set3') is not None else None
    match_df['playerB_set3'] = scoreInfo.find('set3').get('b') if scoreInfo.find('set3') is not None else None
    match_df['playerA_tiebreak3'] = scoreInfo.find('set3').get('tiebreakA') if scoreInfo.find('set3') is not None else None
    match_df['playerB_tiebreak3'] = scoreInfo.find('set3').get('tiebreakB') if scoreInfo.find('set3') is not None else None

    match_df['playerA_set4'] = scoreInfo.find('set4').get('a') if scoreInfo.find('set4') is not None else None
    match_df['playerB_set4'] = scoreInfo.find('set4').get('b') if scoreInfo.find('set4') is not None else None
    match_df['playerA_tiebreak4'] = scoreInfo.find('set4').get('tiebreakA') if scoreInfo.find('set4')is not None else None
    match_df['playerB_tiebreak4'] = scoreInfo.find('set4').get('tiebreakB') if scoreInfo.find('set4') is not None else None

    match_df['playerA_set5'] = scoreInfo.find('set5').get('a') if scoreInfo.find('set5') is not None else None
    match_df['playerB_set5'] = scoreInfo.find('set5').get('b') if scoreInfo.find('set5') is not None else None
    match_df['playerA_tiebreak5'] = scoreInfo.find('set5').get('tiebreakA') if scoreInfo.find('set5') is not None else None
    match_df['playerB_tiebreak5'] = scoreInfo.find('set5').get('tiebreakB') if scoreInfo.find('set5') is not None else None
    
    match_df['playerA_total'] = scoreInfo.find('total').get('a')
    match_df['playerB_total'] = scoreInfo.find('total').get('b')

    return match_df

In [164]:
def getPlayers(root: ET.Element, players: pd.DataFrame):
    """
    Collect player info for each player and store in dataframe
    Args:
        root (ET.Element): root of the XML file for the specific match to parse
        players (pd.DataFrame): df that represents a new row
    Returns:
        players (pd.DataFrame): new single row df with player info
    """
    # To be extended later (or joined with match table), for now player table will only have id, name, country
    contestantsInfo = root.find('matchInfo').find('contestants')
    playerDict = {}

    for contestant in contestantsInfo.iter('contestant'):
        playerDict['player_id'] = contestant.get('id')
        playerDict['player_name'] = contestant.get('name')
        playerDict['player_country'] = contestant.find('country').text

        players = pd.concat([players, pd.DataFrame(playerDict, index=[0])], ignore_index=True)
        

    return players
    

In [166]:
match_df = pd.DataFrame()
tournament_df = pd.DataFrame()
player_df = pd.DataFrame()

for i, root in enumerate(roots):
    matchDict = {}
    tournamentDict = {}
    playerDict = {}

    matchDict = getMatchInfo(root, matchDict)
    tournamentDict, matchDict = getTournamentInfo(root, tournamentDict, matchDict)
    matchDict = getMatchStats(root, matchDict)
    matchDict = getScore(root, matchDict)

    player_df = getPlayers(root, player_df)

    match_df = pd.concat([match_df, pd.DataFrame(matchDict, index=[i])])
    tournament_df = pd.concat([tournament_df, pd.DataFrame(tournamentDict, index=[i])])


match_df = match_df.fillna(0)

In [167]:
match_df.to_csv('match.csv', index=False)
tournament_df.to_csv('tournament.csv', index=False)
player_df.to_csv('player.csv', index=False)

In [ ]:
db_username = 'root'
db_password = 'QSTd1b2s3'
db_host = 'qst-db.cgumu5ik5ntf.eu-west-2.rds.amazonaws.com'
db_port = '3306' 
db_name = 'qst_db'
matches_table_name = 'pre_tennis_matches'
tournaments_table_name = 'pre_tennis_tournaments'
players_table_name = 'pre_tennis_players'

engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

def upload_df_to_sql(df, table_name, engine):
    try:
        df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
        print("DataFrame uploaded successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
    
# upload_df_to_sql(match_df, matches_table_name, engine)
# upload_df_to_sql(tournament_df, tournaments_table_name, engine)
# upload_df_to_sql(player_df, players_table_name, engine)